In [12]:
################################################################################
#############  Purpose                     #####################################
## This code compares mean and variance estimates for Al+ time series 
## (not ratio) for high and low frequency of observations. The low frequency 
## is set to match the sampling rate of Sr. 
## The point is to see if the difference in sampling rates of the two clocks
## has any impact on the estimates of interest. 
################################################################################

import pandas as pd
from decimal import Decimal
import matplotlib.pyplot as plt
import numpy as np
 

In [13]:

################################################################################
#############  Functions for data loading #####################################
################################################################################

## comb data 
def open_ErYb_data(data_path, header=2):
    # keys to read out as string
    key2read = ["MJD", "timer", "SDR:frep_ErYb", "fo_ErYb", "fb_Si_ErYb", "fb_Al_ErYb", "fb_Yb_ErYb"] 
    types = {key: str for key in key2read}
    types["MJD"] = float
 
    # # Read the CSV file
    data = pd.read_csv(data_path, header=1, delimiter="\t", dtype=types, engine="python")
 
    # Convert the strings to Decimal for the given keys
    for k in key2read:
        data[k] = data[k].apply(Decimal)
 
    # reindex data
    data.index = range(len(data))
 
    return data[list(types.keys())]

## Al shift data 
def open_shiftfile_Al(datapath):
    data = pd.read_csv(datapath, header=30, delimiter="\t", dtype={1: str}, engine="python")
 
    # Replace column names
    data.columns = ["MJD", "shift", "IS_GOOD"]
 
    # Change column type from float to bool
    data["IS_GOOD"] = data["IS_GOOD"].apply(lambda x: x == 1.0)
 
    # Put NaN in data["shift"] where data["IS_GOOD"] is 0
    data.loc[~data["IS_GOOD"], "shift"] = np.nan
 
    # Change column type to float
    data["shift"] = data["shift"].apply(float)
 
    return data
 
## Sr shift data 
def open_shiftfile_Sr(datapath):
    data = pd.read_csv(datapath, header=22, delimiter="\t", dtype={1: str}, engine="python")
 
    # Replace column names
    data.columns = ["MJD", "shift", "IS_GOOD"]
 
    # Change column type from float to bool
    data["IS_GOOD"] = data["IS_GOOD"].apply(lambda x: x == 1.0)
 
    # Put NaN in data["shift"] where data["IS_GOOD"] is 0
    data.loc[~data["IS_GOOD"], "shift"] = np.nan
 
    # Change column type to float
    data["shift"] = data["shift"].apply(float)
 
    return data
 
 
################################################################################
#############  Functions to find optical frequencies with comb equation ########
################################################################################
 
# frequency for Al+ clock
def compute_nuAl_ErYb(data):
    data["nuAl"] = -Decimal("105e6") + Decimal("560444") * (Decimal("1e9") + data["SDR:frep_ErYb"]) / Decimal(2) - data["fb_Al_ErYb"]
    data["nuAl"] = Decimal(4) * data["nuAl"]   

# frequency for Sr clock 
def compute_nuSr_ErYb(data):
    data["nuSi"] = -Decimal("105e6") + Decimal("388752") * (Decimal("1e9") + data["SDR:frep_ErYb"]) / Decimal(2) - Decimal("100e6")
    data["nuSr"] = (Decimal("1716882") / Decimal("777577")) * (data["nuSi"] - Decimal("216e6"))


 
################################################################################
#############################  Load data #######################################
################################################################################
 
path = "/Users/smt3/Documents/GitHub/atomic-clock/st-interp/three_clocks/"

# load comb data
data_ErYb = open_ErYb_data(path + "20240813_Deglitched_ErYb_only1.dat")
 
# load Al shift data 
shift_data_Al = open_shiftfile_Al(path + "20240813_Al+_Freq_Shifts_ErYb.dat")

# load Sr shift data
shift_data_Sr = open_shiftfile_Sr(path + "20240813_Sr_Freq_Shifts.dat")
 

In [ ]:
 
################################################################################
###############  get optical frequencies #############################
################################################################################
 
compute_nuSr_ErYb(data_ErYb)
compute_nuAl_ErYb(data_ErYb)
 
 

################################################################################
#########################  Data Processing #####################################
################################################################################

## Extract only "IS_GOOD" data for analysis 
good_condition_al = shift_data_Al["IS_GOOD"] == 1
shift_data_Al_good = shift_data_Al[good_condition_al].reset_index(drop=True)
good_condition_sr = shift_data_Sr["IS_GOOD"] == 1
shift_data_Sr_good = shift_data_Sr[good_condition_sr].reset_index(drop=True)


# ## View frequency of MDJ observations  
# shift_data_Al["MJD_diff"] = shift_data_Al.MJD.diff()

# series_MJD_Al_diff = shift_data_Al["MJD_diff"]
# # plt.hist(series_MJD_Sr_diff.dropna(), bins = 500)
# # plt.xlim(0.00005, 0.00025)
# # plt.show()

# #print(series_MJD_Sr_diff.dtype)
# series_MJD_Al_diff.describe()

#mode MJD diff for Sr: 0.0001, max diff: 0.001224
#mode MJD diff for Al: 0.000012, max diff: 0.018264

common_mjd = data_ErYb["MJD"].astype(float)
nuAl = data_ErYb["nuAl"].astype(float)
nuAl = [Decimal(i) for i in nuAl]


In [ ]:

## Create low sampling frequency observations of Al and distinguish this from the high sampling case
## Do the same for the relevant comb values 

#function to extract element as close to target as possible w/out going over
def lb_extract(target, data):
    inx = 0
    stopper = 1
    while stopper == 1:
        if data[inx] <= target:
            inx += 1
        else:
            return inx  

#function to extract element as close to target as possible w/out going under 
def ub_extract(target, data):
    inx = 1
    stopper = 1
    while stopper == 1:
        if data[len(data)-inx] >= target:
            inx += 1
        else:
            return len(data)-inx  

#### first comb time point:  60535.682346  (# first good Al time point:  60535.6818403)
#### last good Al time point:  60535.9108218  (# last comb time point:  60535.911951)

len_comb = len(common_mjd) 
len_Al = len(shift_data_Al_good['MJD'])                  

comb_high = pd.DataFrame()
comb_end = lb_extract(target = shift_data_Al_good["MJD"][len_Al-1], data = common_mjd)  
comb_high["MJD"] = common_mjd[:comb_end]
comb_high["nuAl"] = nuAl[:comb_end]

al_start = ub_extract(target = common_mjd[0], data = shift_data_Al_good)
shift_data_Al_high = shift_data_Al_good[al_start:]


inx_low = #sample from grid based on min/max and mode mdj of Al w/ rate of 0.0001

comb_low = pd.DataFrame()
shift_data_Al_low = shift_data_Al_good[]



In [ ]:
################################################################################
#########################  Interpolation             ###########################
## Is this even necessary to prove point ? may skip ---
################################################################################
# shiftAl_low = np.interp(comb_low["MJD"], shift_data_Al_low["MJD"], shift_data_Al_low["shift"])
# shiftAl_low = [Decimal(i) for i in shiftAl_low]

# shiftAl_high = np.interp(comb_high["MJD"], shift_data_Al_high["MJD"], shift_data_Al_high["shift"])
# shiftAl_high = [Decimal(i) for i in shiftAl_high]

In [ ]:
################################################################################
#########################  Estimation                ###########################
## TODO: calculate AVAR for each set also 
################################################################################
# frequency corrections
masercorrection = Decimal("-7.36631e-12")
GR_shift_Al = Decimal("-8.114e-16")
GR_shift_sea_level = Decimal("-1798.501e-16")
total_correction_Al = Decimal("1") + GR_shift_Al + GR_shift_sea_level + masercorrection

frequency_Al_ErYb_low = [(i + j) * total_correction_Al for i,j in zip(comb_low["nuAl"], shiftAl_low)]
frequency_Al_ErYb_high = [(i + j) * total_correction_Al for i,j in zip(comb_high["nuAl"], shiftAl_high)]
print("Al+ ave with lower sampling rate", '{:0.5}'.format(np.nanmean(frequency_Al_ErYb_low)), '\n' )
print("Al+ ave with higher sampling rate", '{:0.5}'.format(np.nanmean(frequency_Al_ErYb_high)) )

